In [1]:
print("ok")

ok


In [39]:
%pwd

'c:\\Users\\ravig\\MedicalBot\\Medical-Bot'

In [40]:
import os
os.chdir("../")

In [41]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter


In [42]:
# Extract Data From the PDF File

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [43]:
extracted_data = load_pdf_file(data = "C:/Users/ravig/MedicalBot/Medical-Bot/research/Data")


In [6]:
extracted_data


[Document(metadata={'producer': 'Acrobat Distiller 9.5.5 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2020-12-07T12:26:56+01:00', 'author': 'Gerald', 'moddate': '2020-12-07T12:32:18+01:00', 'title': 'HIV2015_GB_HIV2015_GB', 'source': 'C:\\Users\\ravig\\MedicalBot\\Medical-Bot\\research\\Data\\medicalbook.pdf', 'total_pages': 775, 'page': 0, 'page_label': 'I'}, page_content='MedizinFokusVerlag\nwww.hivbook.com\nHIV2015/2016\nHoffmann Rockstroh|'),
 Document(metadata={'producer': 'Acrobat Distiller 9.5.5 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2020-12-07T12:26:56+01:00', 'author': 'Gerald', 'moddate': '2020-12-07T12:32:18+01:00', 'title': 'HIV2015_GB_HIV2015_GB', 'source': 'C:\\Users\\ravig\\MedicalBot\\Medical-Bot\\research\\Data\\medicalbook.pdf', 'total_pages': 775, 'page': 1, 'page_label': 'II'}, page_content='This project has been supported\nwith unrestriced grants from\nAbbVie\nGilead Sciences\nHEXAL\nJanssen-Cilag\nMSD\nViiV H

In [44]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks



In [45]:
text_chunks = text_split( extracted_data)
print("Length of text chunks:", len(text_chunks))


Length of text chunks: 6277


In [46]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the PDF file
loader = PyPDFLoader("C:/Users/ravig/MedicalBot/Medical-Bot/research/Data/medicalbook.pdf")
extracted_data = loader.load()

# Split the text
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))


Length of text chunks: 6277


In [9]:
text_chunks

[Document(metadata={'producer': 'Acrobat Distiller 9.5.5 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2020-12-07T12:26:56+01:00', 'author': 'Gerald', 'moddate': '2020-12-07T12:32:18+01:00', 'title': 'HIV2015_GB_HIV2015_GB', 'source': 'C:\\Users\\ravig\\MedicalBot\\Medical-Bot\\research\\Data\\medicalbook.pdf', 'total_pages': 775, 'page': 0, 'page_label': 'I'}, page_content='MedizinFokusVerlag\nwww.hivbook.com\nHIV2015/2016\nHoffmann Rockstroh|'),
 Document(metadata={'producer': 'Acrobat Distiller 9.5.5 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2020-12-07T12:26:56+01:00', 'author': 'Gerald', 'moddate': '2020-12-07T12:32:18+01:00', 'title': 'HIV2015_GB_HIV2015_GB', 'source': 'C:\\Users\\ravig\\MedicalBot\\Medical-Bot\\research\\Data\\medicalbook.pdf', 'total_pages': 775, 'page': 1, 'page_label': 'II'}, page_content='This project has been supported\nwith unrestriced grants from\nAbbVie\nGilead Sciences\nHEXAL\nJanssen-Cilag\nMSD\nViiV H

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings


In [48]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [49]:
embedding_model = download_hugging_face_embeddings()

In [13]:
pip install hf_xet

Note: you may need to restart the kernel to use updated packages.


In [14]:
embedding_model = download_hugging_face_embeddings()

In [50]:
from dotenv import load_dotenv
import os

from pinecone import Pinecone, ServerlessSpec

# 1) load your .env
load_dotenv()

# 2) grab the API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) initialize the HTTP client with your environment
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment="us-east1-aws"
)

# 4) create (or connect to) your index
index_name = "medicalbot"

if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"✅ Index '{index_name}' already exists")

# 5) now you can upsert/query vectors via Pinecone
index = pc.Index(index_name)
print(f"🔗 Connected to index '{index_name}'")


✅ Index 'medicalbot' already exists
🔗 Connected to index 'medicalbot'


In [68]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Safely get keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Check if they loaded correctly
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY not found in .env file.")
if not GROQ_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file.")

# Set into environment
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [52]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
)




In [18]:
docsearch

In [53]:
# Create a retriever that only uses “k” (no fetch_k)
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}   # only k is supported here
)

# Now invoke it on your query:
retrieved_docs = retriever.invoke("What is hiv?")

# Or, if you’d rather pull Document objects directly:
docs = retriever.get_relevant_documents("What is hiv?")


In [20]:
docs

[Document(id='bef40be2-8dd0-4d9b-a7cf-4e592cadc471', metadata={'author': 'Gerald', 'creationdate': '2020-12-07T12:26:56+01:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2020-12-07T12:32:18+01:00', 'page': 4.0, 'page_label': 'V', 'producer': 'Acrobat Distiller 9.5.5 (Windows)', 'source': 'C:/Users/ravig/MedicalBot/Medical-Bot/research/Data/medicalbook.pdf', 'title': 'HIV2015_GB_HIV2015_GB', 'total_pages': 775.0}, page_content='Christian Hoffmann, M.D., Ph.D.\nICH Stadtmitte (Infektionsmedizinisches Centrum Hamburg)\nGlockengiesserwall 1 \n20095 Hamburg, Germany \nPhone: + 49 40 2800 4200 \nFax: + 49 40 2800 42020\nhoffmann@ich-hamburg.de\nJürgen K. Rockstroh, M.D., Ph.D.\nDepartment of Medicine I\nUniversity of Bonn\nSigmund-Freud-Strasse 25\n53105 Bonn, Germany\nPhone: + 49 228 287 6558\nFax: + 49 228 287 5034\nJuergen.Rockstroh@ukb.uni-bonn.de\nHIV Medicine is an ever-changing field. The editors and authors of HIV 2015/16'),
 Document(id='1a4e31f0-964f-490c-8d61-65c8df3928

In [54]:
docs[0].page_content

'Christian Hoffmann, M.D., Ph.D.\nICH Stadtmitte (Infektionsmedizinisches Centrum Hamburg)\nGlockengiesserwall 1 \n20095 Hamburg, Germany \nPhone: + 49 40 2800 4200 \nFax: + 49 40 2800 42020\nhoffmann@ich-hamburg.de\nJürgen K. Rockstroh, M.D., Ph.D.\nDepartment of Medicine I\nUniversity of Bonn\nSigmund-Freud-Strasse 25\n53105 Bonn, Germany\nPhone: + 49 228 287 6558\nFax: + 49 228 287 5034\nJuergen.Rockstroh@ukb.uni-bonn.de\nHIV Medicine is an ever-changing field. The editors and authors of HIV 2015/16'

In [56]:
# Create a retriever that only uses “k” (no fetch_k)
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}   # only k is supported here
)

# Now invoke it on your query:
retrieved_docs = retriever.invoke("What is hiv?")

# Or, if you’d rather pull Document objects directly:
docs = retriever.get_relevant_documents("What is hiv?")


In [24]:
%pip install langchain_groq

Note: you may need to restart the kernel to use updated packages.


In [70]:
# Set API key
api_key = "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"

from langchain_groq import ChatGroq
# Initialize the Groq LLM with API key
llm = ChatGroq(
 model_name="llama-3.1-8b-instant",
 temperature=0.4,
 api_key=api_key
)

# Create a retriever that only uses “k” (no fetch_k)
retriever = docsearch.as_retriever(
 search_type="similarity",
 search_kwargs={"k":3} # only k is supported here
)

# Now invoke it on your query:
retrieved_docs = retriever.invoke("What is hiv?")

# Or, if you’d rather pull Document objects directly:
docs = retriever.get_relevant_documents("What is hiv?")

# Combine the retrieved documents and the query into a single prompt
from langchain.prompts import PromptTemplate
prompt_template = """
You are a medical expert. Answer the question based on the provided context.
Context:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(
 template=prompt_template,
 input_variables=["context", "question"]
)

# Combine the retrieved documents into a single context string
context =""
for doc in docs:
 context += doc.page_content + "\n\n"

final_answer = llm.invoke(
 prompt.format(context=context, question="What is hiv?")
)
print(final_answer)

content='Based on the provided context, HIV stands for Human Immunodeficiency Virus. It\'s a type of virus that attacks the body\'s immune system, specifically targeting the CD4 cells (also known as T cells), which play a crucial role in helping the immune system fight off infections.\n\nAs mentioned in the context, the editors and authors of the book "HIV 2015/16" highlight the complex relationship between HIV and the human immune system. They explain that HIV infection leads to a significant weakening of the immune system, making it difficult for the body to fight off other infections and diseases.\n\nIt\'s worth noting that the book "HIV 2015/16" is a medical textbook written by experts in the field of HIV medicine, and it provides in-depth information on the pathogenesis of HIV infection, including the role of the immune system in responding to the virus.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 177, 'prompt_tokens': 368, 'total_tokens': 545, 'c

In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [79]:
response = rag_chain.invoke(
    {
        "input": "Exaplain me like a 5 years ols child. What is Epidemiology?",
    }
)

print(response["answer"])

Epidemiology is a big word, isn't it? 

Epidemiology is like being a detective to find out how many people have a sickness, like HIV. They look at numbers and try to figure out who has it, where they got it, and how to stop it from spreading. It's like solving a puzzle to help keep people healthy!


In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)